# ETL - COVID State Vaccination Data
This project extracts, transforms and loads COVID data from multiple sources into CSVs to analyze and visualize. 
## Data Sources
- **Vaccination Data**: I downloaded several different files of vaccination data from "Our World in Data" - (https://ourworldindata.org/us-states-vaccinations ):
    - us-covid-number-fully-vaccinated.csv
    - us-covid-share-fully-vaccinated.csv
    - us-total-covid-19-vaccine-doses-administered.csv
    - us-state-covid-vaccines-per-100.csv
<br><br>
- **COVID Cases and Deaths**: I downloaded case counts and death metrics from CDC's COVID Data Tracker at https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36 
    - United_States_COVID_19_Cases_and_Deaths_by_State_over_Time.csv
<br>

#### ETL - The process of extracting, transforming, loading and validating this data is detailed in the following steps:
1. [Extract and Transform National and State level Vaccination Data](#Extract-and-Transform-National-and-State-level-Vaccination-Data)
2. [Extract and Transform COVID Cases and Deaths Data](#Extract-and-Transform-COVID-Case-and-Death-Data)



In [1]:
# Set up Dependencies
import pandas as pd
# Import psycopg2 - the DB API 2.0 compliant PostgreSQL driver for Python


## Extract and Transform National and State level Vaccination Data 

In [2]:
# - us-covid-number-fully-vaccinated.csv
#    - us-covid-share-fully-vaccinated.csv
#    - us-total-covid-19-vaccine-doses-administered.csv
#    - us-state-covid-vaccines-per-100.csv
# Loead the files of vaccination data into Pandas dataframes
number_fully_vaccinated_to_load = "Resources/us-covid-number-fully-vaccinated.csv"
share_fully_vaccinated_to_load = "Resources/us-covid-share-fully-vaccinated.csv"
number_doses_administered_to_load = "Resources/us-total-covid-19-vaccine-doses-administered.csv"
number_doses_per_hundred_to_load = "Resources/us-state-covid-vaccines-per-100.csv"

# Read Vaccine data files and store into Pandas DataFrames
nbr_fully_vaccinated_df = pd.read_csv(number_fully_vaccinated_to_load)
shr_fully_vaccinated_df = pd.read_csv(share_fully_vaccinated_to_load)
nbr_doses_administered_df = pd.read_csv(number_doses_administered_to_load)
nbr_doses_per_hundred_df = pd.read_csv(number_doses_per_hundred_to_load)


In [4]:
nbr_fully_vaccinated_df.head()

,Entity,Day,people_fully_vaccinated
0,Alabama,2021-01-12,7270
1,Alabama,2021-01-13,9245
2,Alabama,2021-01-15,13488
3,Alabama,2021-01-19,16346
4,Alabama,2021-01-20,17956


In [5]:
shr_fully_vaccinated_df.head()

,Entity,Day,people_fully_vaccinated_per_hundred
0,Alabama,2021-01-12,0.15
1,Alabama,2021-01-13,0.19
2,Alabama,2021-01-15,0.28
3,Alabama,2021-01-19,0.33
4,Alabama,2021-01-20,0.37


In [6]:
nbr_doses_administered_df.head()

,Entity,Day,total_vaccinations
0,Alabama,2021-01-12,78134
1,Alabama,2021-01-13,84040
2,Alabama,2021-01-14,92300
3,Alabama,2021-01-15,100567
4,Alabama,2021-01-19,130795


In [7]:
nbr_doses_per_hundred_df.head()

,Entity,Day,total_vaccinations_per_hundred
0,Alabama,2021-01-12,1.59
1,Alabama,2021-01-13,1.71
2,Alabama,2021-01-14,1.88
3,Alabama,2021-01-15,2.05
4,Alabama,2021-01-19,2.67


In [9]:
# Remove empty columns before merging
#nbr_fully_vaccinated_df = nbr_fully_vaccinated_df.drop(columns=['Code'])
#shr_fully_vaccinated_df = shr_fully_vaccinated_df.drop(columns=['Code'])
#nbr_doses_administered_df = nbr_doses_administered_df.drop(columns=['Code'])
#nbr_doses_per_hundred_df = nbr_doses_per_hundred_df.drop(columns=['Code'])

# Combine the data into a single dataset 
df1 = pd.merge(nbr_fully_vaccinated_df, shr_fully_vaccinated_df, how="left", on=["Entity","Day"])
df2 = pd.merge(df1,nbr_doses_administered_df, how="left", on=["Entity","Day"])
df3 = pd.merge(df2, nbr_doses_per_hundred_df, how="left", on=["Entity","Day"])

In [11]:
df3.head()

,Entity,Day,people_fully_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations,total_vaccinations_per_hundred
0,Alabama,2021-01-12,7270,0.15,78134,1.59
1,Alabama,2021-01-13,9245,0.19,84040,1.71
2,Alabama,2021-01-15,13488,0.28,100567,2.05
3,Alabama,2021-01-19,16346,0.33,130795,2.67
4,Alabama,2021-01-20,17956,0.37,139200,2.84


In [12]:
# Clean up the merged file: 
# Remove duplicate data that is listed under Federal Agencies (in addition to containing duplicates, it also contains
# many NaNs): Bureau of Prisons, Dept of Defense, Indian Health Svc, Long Term Care and Veterans Health
vaccinations_df = df3.loc[(df3["Entity"] != "Bureau of Prisons") &
                          (df3["Entity"] != "Dept of Defense") &
                          (df3["Entity"] != "Indian Health Svc") &
                          (df3["Entity"] != "Long Term Care") &
                          (df3["Entity"] != "Veterans Health"), :].copy()

# Replace remaining NaN values with zeros - these primarily occurred on the first day of data collection for some states.
vaccinations_df.fillna(value=0, inplace=True)

# Change the columns back to integers (fillna added an unnecessary decimal position)
vaccinations_df['total_vaccinations'] = vaccinations_df['total_vaccinations'].astype(int) 
vaccinations_df['total_vaccinations_per_hundred'] = vaccinations_df['total_vaccinations_per_hundred'].astype(int)


In [13]:
# Restructure the data before finalizing it
# Change column name from Entity to State to better reflect the content of the final, cleaned up dataframe. 
vaccinations_df.rename(columns={'Entity':'state_name', 'Date':'date_administered' }, 
                 inplace=True)

# Remove the rows of national (state_name ="US") data into its own dataframe to create 2 tables: US_vaccinations
# and State_vaccinations.
# Important note: the national and state numbers aren't always the same, because of the way that the different
# jurisdictions report their data and how the CDC cross-checks and totals it up so I am preserving that difference
# by creating two separate tables: US_vaccinations and state_vaccinations:
US_vaccinations_df = vaccinations_df.loc[(vaccinations_df["state_name"] == "United States"), :].copy()
US_vaccinations_df = US_vaccinations_df.drop(columns=['state_name'])
US_vaccinations_df.reset_index(drop=True, inplace=True)

state_vaccinations_df = vaccinations_df.loc[(vaccinations_df["state_name"] != "United States"), :].copy()
state_vaccinations_df.reset_index(drop=True, inplace=True)


In [14]:
# Write the merged/cleaned up files to new csv files for backup purposes 
US_vaccinations_df.to_csv(r"Resources\US_vaccinations.csv", index = False, encoding="utf-8")
state_vaccinations_df.to_csv(r"Resources\State_Vaccinations.csv", index = False, encoding="utf-8")

## Extract and Transform COVID Case and Death Data 

In [15]:
# Load the file of case and death data, as well as the state cross-reference file, into Pandas dataframes
cases_and_deaths_to_load = "Resources/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
state_xref_to_load = "Resources/State-XRef.csv"

# Read COVID cases & deaths data file and store into Pandas DataFrame
raw_cases_deaths_df = pd.read_csv(cases_and_deaths_to_load)

# Read State cross-reference data
state_xref_df = pd.read_csv(state_xref_to_load)


In [16]:
# Add the state_name to the cases&deaths dataframe so that we will be able to join COVID cases and deaths 
# with vaccinations when querying the DB. 
df1 = pd.merge(raw_cases_deaths_df, state_xref_df, how="left", on=["state"])

# Remove unnecessary columns
raw_cases_deaths_df = df1.drop(columns=['conf_cases','prob_cases','pnew_case',
                                                        'conf_death','prob_death','pnew_death',
                                                       'created_at','consent_cases','consent_deaths','state'])
# reorder the columns
raw_cases_deaths_df = raw_cases_deaths_df[["submission_date", "state_name", "tot_cases", "new_case", "tot_death","new_death"]]

In [17]:
# Add the NYC totals to the NY totals and remove the NYC rows - CDC continued to track theses separately
# since the US pandemic really ramped up in a big way in NYC at the beginning.
# Where the submission_date is the same, add the NCY values to the values in the NY columns
# Pull out the NY rows and the NYC rows into separate DFs for merging and totaling 
NY_cases_deaths_df = raw_cases_deaths_df.loc[(raw_cases_deaths_df["state_name"] == "New York"), :].copy()
NY_cases_deaths_df.reset_index(drop=True, inplace=True)

NYC_cases_deaths_df = raw_cases_deaths_df.loc[(raw_cases_deaths_df["state_name"] == "New York City"), :].copy()
NYC_cases_deaths_df.reset_index(drop=True, inplace=True)

other_states_df = raw_cases_deaths_df.loc[(raw_cases_deaths_df["state_name"] != "New York City") &
                                          (raw_cases_deaths_df["state_name"] != "New York"), :].copy()
other_states_df.reset_index(drop=True, inplace=True)

# Merge the NY & NYC data
merged_df = pd.merge(NY_cases_deaths_df, NYC_cases_deaths_df, how="left", on=["submission_date"])

# Add the NY & NYC totals together and create a new DF with the daily totals aggregated
submission_date = merged_df['submission_date']
tot_cases = merged_df['tot_cases_x'] + merged_df['tot_cases_y']
new_case = merged_df['new_case_x'] + merged_df['new_case_y']
tot_death = merged_df['tot_death_x'] + merged_df['tot_death_y']
new_death = merged_df['new_death_x'] + merged_df['new_death_y']
merged_ny_df = pd.DataFrame({'submission_date': submission_date,
                                     'state_name':'New York',
                                     'tot_cases': tot_cases,
                                     'new_case': new_case,
                                     'tot_death': tot_death,
                                     'new_death': new_death                                     
                                    })

# Add the new single combined NY/NYC rows back together with the rows from the other states
state_cases_deaths = pd.concat([merged_ny_df, other_states_df])

# Clean up the column names  
state_cases_deaths.rename(columns={'new_case':'new_cases', 'tot_death':'tot_deaths','new_death':'new_deaths' }, 
                 inplace=True)

# Write the merged/cleaned up file to a new csv file for backup purposes 
state_cases_deaths.to_csv(r"Resources\state_cases_deaths.csv", index = False, encoding="utf-8")
